In [1]:
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv("../data/cleaned_transactions.csv")
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])
df.head()


,TransactionID,CustomerID,AccountID,AccountType,TransactionType,Product,Firm,Region,Manager,TransactionDate,TransactionAmount,AccountBalance,RiskScore,CreditRating,TenureMonths,Year,Month,YearMonth
0,33,CUST6549,ACC12334,credit,withdrawal,Savings Account,Firm C,Central,Manager 1,2023-10-21,87480.05448,74008.43310,0.729101,319,200,2023,10,2023-10
1,177,CUST2942,ACC52650,credit,withdrawal,Home Loan,Firm A,East,Manager 3,2023-06-20,20315.74505,22715.83590,0.472424,692,47,2023,6,2023-06
2,178,CUST6776,ACC45101,current,deposit,Personal Loan,Firm C,South,Manager 3,2023-01-02,10484.57165,42706.09210,0.648784,543,109,2023,1,2023-01
3,173,CUST2539,ACC88252,current,withdrawal,Mutual Fund,Firm A,Central,Manager 2,2023-07-25,45122.27373,114176.56870,0.734832,430,103,2023,7,2023-07
4,67,CUST2626,ACC21878,savings,withdrawal,Home Loan,Firm C,Central,Manager 4,2023-07-25,42360.79878,17863.02644,0.289304,468,234,2023,7,2023-07


In [3]:
withdrawals = df[df['TransactionType'] == 'withdrawal']


In [4]:
large_withdrawals = withdrawals[
    withdrawals['TransactionAmount'] > withdrawals['TransactionAmount'].quantile(0.90)
]


In [5]:
frequent_large_withdrawals = (
    large_withdrawals
    .groupby('AccountID')
    .size()
    .reset_index(name='Large_Withdrawal_Count')
    .sort_values('Large_Withdrawal_Count', ascending=False)
)

frequent_large_withdrawals.head()


,AccountID,Large_Withdrawal_Count
13,ACC60432,2
14,ACC66086,2
0,ACC16241,1
2,ACC31539,1
1,ACC17688,1


In [6]:
frequent_large_withdrawals.to_excel(
    "../excel_outputs/Task4_Frequent_Large_Withdrawals.xlsx",
    index=False
)


In [7]:
balance_volatility = (
    df
    .groupby('AccountID')['AccountBalance']
    .std()
    .reset_index(name='Balance_StdDev')
    .sort_values('Balance_StdDev', ascending=False)
)

balance_volatility.head()


,AccountID,Balance_StdDev
25,ACC21878,70517.525297
64,ACC34821,61454.350302
5,ACC11837,60694.391957
117,ACC58667,58637.344639
138,ACC71938,56593.572697


In [8]:
balance_volatility.to_excel(
    "../excel_outputs/Task4_Balance_Volatility.xlsx",
    index=False
)


In [9]:
Q1 = df['TransactionAmount'].quantile(0.25)
Q3 = df['TransactionAmount'].quantile(0.75)
IQR = Q3 - Q1


In [10]:
anomalies = df[
    (df['TransactionAmount'] < Q1 - 1.5 * IQR) |
    (df['TransactionAmount'] > Q3 + 1.5 * IQR)
]

anomalies.head()


,TransactionID,CustomerID,AccountID,AccountType,TransactionType,Product,Firm,Region,Manager,TransactionDate,TransactionAmount,AccountBalance,RiskScore,CreditRating,TenureMonths,Year,Month,YearMonth
266,14,CUST3015,ACC21719,loan,deposit,Savings Account,Firm D,North,Manager 3,2024-05-22,-30721.24789,113801.0737,0.378442,360,222,2024,5,2024-05


In [11]:
anomalies.to_excel(
    "../excel_outputs/Task4_Transaction_Anomalies.xlsx",
    index=False
)


In [12]:
customer_behavior = (
    df
    .groupby('CustomerID')
    .agg(
        Total_Transactions=('TransactionID', 'count'),
        Avg_Balance=('AccountBalance', 'mean'),
        Balance_StdDev=('AccountBalance', 'std')
    )
    .reset_index()
)

customer_behavior.head()


,CustomerID,Total_Transactions,Avg_Balance,Balance_StdDev
0,CUST1042,8,80435.167530,28905.348524
1,CUST1114,8,60127.893845,39795.711402
2,CUST1121,4,83213.472732,16557.362431
3,CUST1189,3,63648.890237,27282.974000
4,CUST1223,2,88186.197605,48462.228205


In [13]:
suspicious_customers = customer_behavior[
    (customer_behavior['Total_Transactions'] > customer_behavior['Total_Transactions'].quantile(0.90)) |
    (customer_behavior['Balance_StdDev'] > customer_behavior['Balance_StdDev'].quantile(0.90))
]

suspicious_customers.head()


,CustomerID,Total_Transactions,Avg_Balance,Balance_StdDev
0,CUST1042,8,80435.167530,28905.348524
1,CUST1114,8,60127.893845,39795.711402
10,CUST1547,2,42375.457815,56481.575397
12,CUST1569,3,54201.293253,68939.751930
14,CUST1644,8,71087.274111,25707.812074


In [14]:
suspicious_customers.to_excel(
    "../excel_outputs/Task4_Suspicious_Customers.xlsx",
    index=False
)


In [ ]:
#with pd.ExcelWriter(
 #   "../excel_outputs/Financial_Risk_Analysis_Task4_FINAL.xlsx",
  #  engine="openpyxl"
#) as writer:

 #   frequent_large_withdrawals.to_excel(
  #      writer,
   #     sheet_name="Frequent_Large_Withdrawals",
    #    index=False
    #)

    #balance_volatility.to_excel(
     #   writer,
      #  sheet_name="Balance_Volatility",
       # index=False
    #)

    #anomalies.to_excel(
     #   writer,
      #  sheet_name="Transaction_Anomalies",
       # index=False
    #)

    #suspicious_customers.to_excel(
     #   writer,
      #  sheet_name="Suspicious_Customers",
       # index=False
    #)

#print("Financial_Risk_Analysis_Task4_FINAL.xlsx created successfully")


## Insights:
Frequent large withdrawals and high balance volatility are strong indicators of financial risk.
Anomaly detection techniques helped identify unusual transaction patterns effectively.
Risk-based identification supports proactive monitoring and fraud prevention.